# Libraries

In [1]:
__author__ = "Aiello Davide"
import random
import logging
from collections import namedtuple
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor

# Nim Class

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

# Task 2: Evolved rules

In [4]:
POPULATION_SIZE = 20           
OFFSPRING_SIZE = 10         

NUM_GENERATIONS = 100        

TOURNAMENT_SIZE = 2
GENETIC_OPERATOR_RANDOMNESS = 0.7
logging.getLogger().setLevel(logging.INFO)

NUM_MATCHES = 100
NIM_SIZE = 11
k = 6

In [5]:
def shortest_row(state: Nim) -> Nimply:
    row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    if(state.rows[row] > k):
       num_objects = random.randint(1, k)
    else:
       num_objects = state.rows[row]
    return Nimply(row, num_objects)

def Davide_strategy(state: Nim) -> Nimply:
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    if any([True for i in possible_moves if i[1] > 1]):
        obj = 0
        while(obj == 0):
            row_num = random.randint(0, len(state.rows) - 1)
            if state.rows[row_num] > 0:
                obj = max([i[1] for i in possible_moves if i[0] == row_num], key=lambda i:i)
                if obj > k:
                    obj = k
                ply = Nimply(row_num, obj)
    else: 
        ply = None
        while ply == None or ply[1] > k:
            ply = Nimply(*possible_moves[random.randint(0, len(possible_moves) - 1)])
    return ply

def GabriG_strategy(state: Nim) -> Nimply:
    row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    if(state.rows[row] <= k):
        num_objects = state.rows[row]
    else:
        if(state.rows[row] > (k*2)):
            num_objects = k
        else:
            num_objects =state.rows[row] - k
    return Nimply(row, num_objects)

def longest_row(state: Nim) -> Nimply:
    row = max((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    if(state.rows[row] > k):
       num_objects = random.randint(1, k)
    else:
       num_objects = state.rows[row]
    return Nimply(row, num_objects)

def pure_random(state: Nim) -> Nimply:        
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0]) 
    num_objects = None
    while num_objects == None or num_objects > k:    
        num_objects = random.randint(1, state.rows[row])                     
    return Nimply(row, num_objects)

Individual = namedtuple("Individual", ["genome", "fitness"])

def tournament(population, tournament_size=TOURNAMENT_SIZE):          
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness) 

def reweigth_prob(g):
    actual_sum = sum(g)
    for i in range(len(g)):
        if actual_sum == 0:
            break
        g[i] = g[i] / actual_sum
    return g

def uniform_cross_over(g1, g2, strategy: list):
    new_genoma = []
    for i in range(len(strategy)):
        if i%2:
            new_genoma.append(g1[i])
        else:
            new_genoma.append(g2[i])
    new_genoma = reweigth_prob(new_genoma)
    return tuple(new_genoma)

def mutation(g, strategy: list):                                
    point = random.randint(0, len(strategy) - 1)
    if random.random() < .5:   
        g =  g[:point] + (g[point] +.05,) + g[point + 1 :]     
    else:
        g = g[:point] + (g[point] -.05,) + g[point + 1 :]
    g = [i for i in g]
    return tuple(reweigth_prob(g))

def compute_fitness(genome, strategy: list):   
    s =  max(enumerate(genome), key=lambda x: x[1])[0]                     
    won = 0
    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE, k)
        player = 0
        while nim:
            if player == 0:
                ply = pure_random(nim)
            else:
                ply = strategy[s](nim)
            nim.nimming(ply)
            player = 1 - player
        winner = 1 - player
        if winner == 1:
            won += 1
    return won / NUM_MATCHES

In [6]:
def create_population(strategy: list): 
   population = list()
   for genome in [tuple(1/len(strategy) for _ in range(len(strategy)))]:    
      population.append(Individual(genome, compute_fitness(genome, strategy))) 
   return population

In [7]:
def play():
    strategy = [shortest_row, Davide_strategy, GabriG_strategy, longest_row, pure_random]
    population = create_population(strategy)
    fitness_log = [(0, i.fitness) for i in population]  
    best_fit = 0
    for g in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            if random.random() < GENETIC_OPERATOR_RANDOMNESS:                         
                p = tournament(population)                  
                o = mutation(p.genome, strategy)                    
            else:                                          
                p1 = tournament(population)                 
                p2 = tournament(population)
                o = uniform_cross_over(p1.genome, p2.genome, strategy)          
            f = compute_fitness(o, strategy)                                      
            fitness_log.append((g + 1, f))                     
            offspring.append(Individual(o, f))                 
        population += offspring    
        population = sorted(population, key=lambda i: i[1], reverse=True)[:POPULATION_SIZE]
    return strategy[max(enumerate(population[0][0]), key=lambda x: x[1])[0]], population[0][1] * 100


# Main

In [8]:
logging.getLogger().setLevel(logging.DEBUG)

strat, fitness = play()
logging.info(f"The best strategy is {strat.__qualname__} with {fitness}% winrate (fitness)")

KeyboardInterrupt: 

# Alternative 

In [18]:
k = 100


def reweigth_prob(g):
    actual_sum = sum(g)
    for i in range(len(g)):
        if actual_sum == 0:
            break
        g[i] = g[i] / actual_sum
    return g

def uniform_cross_over(g1, g2):
    new_genoma = dict()
    new_genoma['alpha'] = g1['alpha']
    new_genoma['beta'] = g2['beta']
    return new_genoma

def mutation(g): 
    tweak = .05                       
    point = random.random() 
    if point < .25:
        if g['alpha'] + tweak < 0.6:
             g['alpha'] += tweak
        else:
             g['alpha'] -= tweak
    if  0.25 <= point < .5:
        if g['alpha'] - tweak > 0:
             g['alpha'] -= tweak
        else:
             g['alpha'] += tweak
    tweak = tweak*1.5
    if  0.5 <= point < .75:
        if g['beta'] + tweak < 1:
             g['beta'] += tweak
        else:
             g['beta'] -= tweak
    if  0.75 <= point < 1:
        if g['beta'] - tweak > 0:
             g['beta'] -= tweak
        else:
             g['beta'] += tweak
    return g


def build_strategy(state: Nim, g: namedtuple) -> Nimply: 
    if g['alpha'] <= .2:
        row = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    if g['alpha'] > .2 and g['alpha'] <= .4:
        row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    if g['alpha'] <= .6:
        row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    if g['beta'] <= .25:
        objs = state.rows[row]
    if g['beta'] > .25 and g['beta'] <= .50:
        objs = 1
    if g['beta'] > .50 and g['beta'] <= .75:
        objs = state.rows[row]//2
    if g['beta'] <= 1:
        objs = random.randint(1, state.rows[row])
    return Nimply(row, objs)


def create_population(): 
   population = list()
   for genome in [{"alpha": .3, "beta": .5} for _ in range(POPULATION_SIZE)]:    
      population.append(Individual(genome, compute_fitness(genome))) 
   return population

def compute_fitness(genome):   
    s =  max(enumerate(genome), key=lambda x: x[1])[0]                     
    won = 0
    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE, k)
        player = 0
        while nim:
            if player == 0:
                ply = pure_random(nim)
            else:
                ply = build_strategy(nim, genome)
            nim.nimming(ply)
            player = 1 - player
        winner = 1 - player
        if winner == 1:
            won += 1
    return won / NUM_MATCHES


population = create_population()
fitness_log = [(0, i.fitness) for i in population]  
for g in range(NUM_GENERATIONS):
    offspring = list()
    for i in range(OFFSPRING_SIZE):
        if random.random() < GENETIC_OPERATOR_RANDOMNESS:                         
            p = tournament(population)                  
            o = mutation(p.genome)                    
        else:                                          
            p1 = tournament(population)                 
            p2 = tournament(population)
            o = uniform_cross_over(p1.genome, p2.genome)          
        f = compute_fitness(o)                                      
        fitness_log.append((g + 1, f))                     
        offspring.append(Individual(o, f))                 
    population += offspring    
    population = sorted(population, key=lambda i: i[1], reverse=True)[:POPULATION_SIZE]
best_ind = population[0]
print(f"the best winrate is {best_ind[1]*100} with paramter: alpha {best_ind[0]['alpha']} - beta {best_ind[0]['beta']}")

the best winrate is 68.0 with paramter: alpha 0.3 - beta 0.7249999999999999
